In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
import os

In [2]:
path = r'C:\Users\user\Documents\PROC\Weekly\2025\11. November\11 November Week 3'
po_file = path + r'\PO.xlsx'
rfm_file = path + r'\RFM.xlsx'

datestart = '14-11-2025'
dateend = '21-11-2025'


In [3]:
def process_procurement_data(po_file, rfm_file, datestart, dateend):
    """Process procurement data for weekly reporting and save to Excel."""

    # Load original files
    df_po_original = pd.read_excel(po_file)
    df_rfm_original = pd.read_excel(rfm_file)

    # Make working copies
    df_po = df_po_original.copy()
    df_rfm = df_rfm_original.copy()

    # Define department mappings
    OBI = ['Rona / Joko', 'Joko', 'Victo', 'Rakan', 'Rona Justhafist', 'Rona / Victo / Rakan / Joko']
    LAR = ['Fairus / Irwan', 'Fairus Mubakri', 'Irwan', 'Ady', 'Fairus / Ady']
    HO = ['Linda / Puji / Syifa R / Stheven', 'Syifa Ramadhani', 'Syifa Alifia', 'Rizal Agus Fianto',
          'Auriel', 'Puji Astuti', 'Linda Permata Sari']

    # Define exclusions
    exclude_category = ['Jasa Logistik', 'Jasa/Service', 'Kontrak', 'Solar']
    exclude_requisition_type = ['Consignment']
    exclude_department = ['test']

    # Load Normalisasi file for updates
    try:
        picnorm = pd.read_excel(r'C:\Users\user\Documents\PROC\Monthly - Weekly\####1Normalisasi\Normalisasi.xlsx', sheet_name='RFM')
        picnorm_indexed = picnorm.drop_duplicates(subset='Requisition Number').set_index('Requisition Number')
    except FileNotFoundError:
        print("Normalisasi.xlsx not found. Skipping data enrichment.")
        picnorm_indexed = None

    # Apply data enrichment from Normalisasi file to both PO and RFM DataFrames
    if picnorm_indexed is not None:
        # Process df_po
        df_po['Updated Requisition Approved Date'] = df_po['Requisition Number'].map(picnorm_indexed.get('Updated Requisition Approved Date'))
        df_po['Updated Requisition Required Date'] = df_po['Requisition Number'].map(picnorm_indexed.get('Updated Requisition Required Date'))
        df_po['Background Update'] = df_po['Requisition Number'].map(picnorm_indexed.get('Background Update'))
        df_po['used_approved_date'] = df_po['Updated Requisition Approved Date'].fillna(df_po['Requisition Approved Date'])
        
        # Process df_rfm
        df_rfm['Updated Requisition Approved Date'] = df_rfm['Requisition Number'].map(picnorm_indexed.get('Updated Requisition Approved Date'))
        df_rfm['Updated Requisition Required Date'] = df_rfm['Requisition Number'].map(picnorm_indexed.get('Updated Requisition Required Date'))
        df_rfm['Background Update'] = df_rfm['Requisition Number'].map(picnorm_indexed.get('Background Update'))
        df_rfm['used_approved_date'] = df_rfm['Updated Requisition Approved Date'].fillna(df_rfm['Requisition Approved Date'])
    else:
        # If no Normalisasi file, use original dates
        df_po['used_approved_date'] = df_po['Requisition Approved Date']
        df_po['used_required_date'] = df_po['Requisition Required Date']
        df_rfm['used_approved_date'] = df_rfm['Requisition Approved Date']
        df_rfm['used_required_date'] = df_rfm['Requisition Required Date']
        df_po['Updated Requisition Approved Date'] = np.nan
        df_po['Updated Requisition Required Date'] = np.nan
        df_po['Background Update'] = np.nan
        df_rfm['Updated Requisition Approved Date'] = np.nan
        df_rfm['Updated Requisition Required Date'] = np.nan
        df_rfm['Background Update'] = np.nan

    # Base filters
    base_filter_po = (
        ~df_po['Item Category'].isin(exclude_category) &
        ~df_po['Requisition Type'].isin(exclude_requisition_type) &
        ~df_po['Department'].str.lower().isin([dept.lower() for dept in exclude_department])
    )

    base_filter_rfm = (
        ~df_rfm['Item Category'].isin(exclude_category) &
        ~df_rfm['Requisition Type'].isin(exclude_requisition_type) &
        ~df_rfm['Project'].str.lower().isin([dept.lower() for dept in exclude_department])
    )

    # Convert date columns
    df_po['PO Approval Date'] = pd.to_datetime(df_po['PO Approval Date'], errors='coerce')
    df_po['used_approved_date'] = pd.to_datetime(df_po['used_approved_date'], errors='coerce')
    df_rfm['used_approved_date'] = pd.to_datetime(df_rfm['used_approved_date'], errors='coerce')
    df_po['Updated Requisition Approved Date'] = pd.to_datetime(df_po['Updated Requisition Approved Date'], errors='coerce')
    df_po['Updated Requisition Required Date'] = pd.to_datetime(df_po['Updated Requisition Required Date'], errors='coerce')
    df_rfm['Updated Requisition Approved Date'] = pd.to_datetime(df_rfm['Updated Requisition Approved Date'], errors='coerce')
    df_rfm['Updated Requisition Required Date'] = pd.to_datetime(df_rfm['Updated Requisition Required Date'], errors='coerce')

    # Date filters
    datestart = pd.to_datetime(datestart, format='%d-%m-%Y')
    dateend = pd.to_datetime(dateend, format='%d-%m-%Y')

    # Filtered data, using 'used_approved_date'
    New_RFMfromPO = df_po[base_filter_po &
                          (df_po['used_approved_date'] >= datestart) &
                          (df_po['used_approved_date'] <= dateend)]

    Inprocess_PO = df_po[base_filter_po &
                         ((df_po['PO Approval Date'].isna()) |
                          (df_po['PO Approval Date'] > dateend))]

    PO_Approved = df_po[base_filter_po &
                        (df_po['PO Approval Date'] >= datestart) &
                        (df_po['PO Approval Date'] <= dateend)]

    New_RFMfromRFM = df_rfm[base_filter_rfm &
                            (df_rfm['Requisition Status'] == 'Approve') &
                            (df_rfm['used_approved_date'] >= datestart) &
                            (df_rfm['used_approved_date'] <= dateend)]

    Inprocess_RFM = df_rfm[base_filter_rfm &
                           (df_rfm['Requisition Status'] == 'Approve') &
                           (df_rfm['used_approved_date'] <= dateend)]

    # Helper: assign unique department only
    def assign_department(df):
        def get_dept(name):
            if pd.isna(name): return None
            for dept, names in [('OBI', OBI), ('LAR', LAR), ('HO', HO)]:
                if any(n in name for n in names):
                    return dept
            return None
        df = df.copy()
        df['Department_Assigned'] = df['Procurement Name'].apply(get_dept)
        return df

    # Helper: split dataframe into department-specific
    def split_by_department(df):
        result = {}
        for dept in ['OBI', 'LAR', 'HO']:
            df_dept = df[df['Department_Assigned'] == dept].drop(columns='Department_Assigned')
            if not df_dept.empty:
                result[dept] = df_dept
        return result

    # Prepare PO results
    po_results = {
        'PO_Approved': assign_department(PO_Approved),
        'New_RFMfromPO': assign_department(New_RFMfromPO),
        'Inprocess_PO': assign_department(Inprocess_PO)
    }

    # Prepare RFM results
    rfm_results = {
        'New_RFMfromRFM': assign_department(New_RFMfromRFM),
        'Inprocess_RFM': assign_department(Inprocess_RFM)
    }

    # === Save PO ===
    with pd.ExcelWriter(os.path.join(path, 'data_PO_Weekly.xlsx'), engine='xlsxwriter') as writer:
        df_po_original.to_excel(writer, sheet_name='Sheet', index=False)  # Save raw input
        for base_name, df in po_results.items():
            # Base export
            df.drop(columns='Department_Assigned').to_excel(writer, sheet_name=base_name[:31], index=False)
            # Dept export
            dept_split = split_by_department(df)
            for dept, df_dept in dept_split.items():
                df_dept.to_excel(writer, sheet_name=f"{base_name}_{dept}"[:31], index=False)

    # === Save RFM ===
    with pd.ExcelWriter(os.path.join(path, 'data_RFM_Weekly.xlsx'), engine='xlsxwriter') as writer:
        df_rfm_original.to_excel(writer, sheet_name='Sheet', index=False)  # Save raw input
        for base_name, df in rfm_results.items():
            # Base export
            df.drop(columns='Department_Assigned').to_excel(writer, sheet_name=base_name[:31], index=False)
            # Dept export
            dept_split = split_by_department(df)
            for dept, df_dept in dept_split.items():
                df_dept.to_excel(writer, sheet_name=f"{base_name}_{dept}"[:31], index=False)

    return {**po_results, **rfm_results}

In [4]:
# Run processing and save to Excel
results = process_procurement_data(po_file, rfm_file, datestart, dateend)

In [5]:
from openpyxl import load_workbook
from openpyxl.styles import PatternFill
from openpyxl.utils import get_column_letter  # <--- REQUIRED Import

def style_and_reorder_excel_by_process(path):
    wb = load_workbook(path)
    
    # CRITICAL FIX: Define the is_po_file variable based on the path
    is_po_file = 'data_PO_Weekly.xlsx' in os.path.basename(path) 
    
    base_names = [
        'PO_Approved', 'New_RFMfromPO', 'Inprocess_PO',
        'New_RFMfromRFM', 'Inprocess_RFM'
    ]

    yellow_fill = PatternFill(start_color='FFFF00', end_color='FFFF00', fill_type='solid')

    ordered_sheets = []

    if 'Sheet' in wb.sheetnames:
        ordered_sheets.append('Sheet')

    for base in base_names:
        ordered_sheets += [s for s in wb.sheetnames if s.startswith(base) and s not in ordered_sheets]

    for sheet in wb.worksheets:
        title = sheet.title
        
        # --- PR-PO CALCULATION LOGIC ---
        # Only runs if it's the PO file AND the sheet is PO_Approved or a department split
        if is_po_file and title.startswith('PO_Approved'):
            
            # Find the required column indices
            headers = [sheet.cell(row=1, column=c).value for c in range(1, sheet.max_column + 1)]
            
            try:
                # Find indices (1-based) for the date columns:
                used_approved_date_col_idx = headers.index('used_approved_date') + 1
                po_approval_date_col_idx = headers.index('PO Approval Date') + 1
                
                # Convert indices to Excel column letters
                used_approved_date_col_letter = get_column_letter(used_approved_date_col_idx)
                po_approval_date_col_letter = get_column_letter(po_approval_date_col_idx)
                
                # Determine the position of the new column
                new_col_idx = sheet.max_column + 1
                
                # Set Header in the new column
                sheet.cell(row=1, column=new_col_idx, value='PR-PO')
                
                # Insert formula into every data row (starting from row 2)
                for row_idx in range(2, sheet.max_row + 1):
                    # Formula: = [PO Approval Date] - [used_approved_date]
                    po_cell = f"{po_approval_date_col_letter}{row_idx}"
                    used_approved_cell = f"{used_approved_date_col_letter}{row_idx}"
                    
                    formula = f"={po_cell}-{used_approved_cell}"
                    sheet.cell(row=row_idx, column=new_col_idx, value=formula)
                
            except ValueError as e:
                print(f"Skipping PR-PO calculation for sheet {title}: Required date column not found.")

        # --- TAB COLORING LOGIC ---
        if title.startswith('PO_Approved'):
            sheet.sheet_properties.tabColor = '00FF00'  # Green
        elif title.startswith('New_RFMfromPO') or title.startswith('New_RFMfromRFM'):
            sheet.sheet_properties.tabColor = '3399FF'  # Blue
        elif title.startswith('Inprocess_PO') or title.startswith('Inprocess_RFM'):
            sheet.sheet_properties.tabColor = 'FFFF00'  # Yellow

        # --- COLUMN HIGHLIGHTING LOGIC ---
        for col_idx, col in enumerate(sheet.iter_cols(min_row=2), start=1):
            header = sheet.cell(row=1, column=col_idx).value
            should_highlight = False

            if title.startswith("PO_Approved") and header == "PO Approval Date":
                should_highlight = True
            elif title.startswith("New_RFMfromPO") and header == "Requisition Approved Date":
                should_highlight = True
            elif title.startswith("Inprocess_PO") and header in ["PO Approval Date", "Requisition Approved Date"]:
                should_highlight = True
            elif title.startswith("New_RFMfromRFM") and header == "Requisition Approved Date":
                should_highlight = True
            elif title.startswith("Inprocess_RFM") and header == "Requisition Approved Date":
                should_highlight = True

            if should_highlight:
                for cell in col:
                    cell.fill = yellow_fill
                    
    # --- REORDERING AND SAVING ---
    wb._sheets = [wb[s] for s in ordered_sheets if s in wb.sheetnames]

    wb.save(path)

In [6]:
style_and_reorder_excel_by_process(os.path.join(path, 'data_PO_Weekly.xlsx'))
style_and_reorder_excel_by_process(os.path.join(path, 'data_RFM_Weekly.xlsx'))